#Installing neccessary libraries

In [ ]:
%pip -q install google-cloud-bigquery
%pip install pandas-gbq -U
%pip install catboost
%pip install pmdarima
%pip install prophet 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importing neccessary libraries 

In [ ]:
import os 
from google.cloud import bigquery 
import pandas as pd 
import pandas_gbq 
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split 
import plotly.graph_objs as go 
from prophet import Prophet 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px 


# Setting up BigQuery client


In [ ]:
key_file = 'oreilly_demo_key.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_file
client = bigquery.Client()

In [ ]:
import json 

f = open(key_file)
data = json.load(f)
project_id = data['project_id']
f.close()


#Querying the google analytics table 

In [ ]:
query1 = """
SELECT 
date, totals.visits, 
hits.hitNumber, totals.transactionRevenue, 
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, 
UNNEST(hits) as hits 
WHERE totals.transactionRevenue >= 10
LIMIT 10 
"""

df = pandas_gbq.read_gbq(query1, project_id = project_id)
df.head()


Downloading: 100%|██████████|


,date,visits,hitNumber,transactionRevenue
0,20170402,1,1,8990000
1,20170402,1,2,8990000
2,20170402,1,3,8990000
3,20170402,1,4,8990000
4,20170402,1,5,8990000


In [ ]:
query1 = """
SELECT 
date, 
totals.transactionRevenue, 
totals.pageviews, 
totals.visits, 
device.browser,
geonetwork.country,
trafficSource.source, 
hits.hitNumber, 
fullVisitorId,
visitId,
visitNumber,

FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, 
UNNEST(hits) as hits
WHERE 
totals.transactionRevenue >= 0 AND 
_TABLE_SUFFIX BETWEEN '20170101' AND '20170731'

LIMIT 10
        """

df = pandas_gbq.read_gbq(query1, project_id=project_id)
df.head()

Downloading: 100%|██████████|


,date,transactionRevenue,pageviews,visits,browser,country,source,hitNumber,fullVisitorId,visitId,visitNumber
0,20170130,87990000,14,1,Chrome,United States,(direct),1,4307411610497308919,1485812030,3
1,20170130,87990000,14,1,Chrome,United States,(direct),2,4307411610497308919,1485812030,3
2,20170130,87990000,14,1,Chrome,United States,(direct),3,4307411610497308919,1485812030,3
3,20170130,87990000,14,1,Chrome,United States,(direct),4,4307411610497308919,1485812030,3
4,20170130,87990000,14,1,Chrome,United States,(direct),5,4307411610497308919,1485812030,3


In [ ]:
print(df.columns)

Index(['visitorId', 'visitNumber', 'visitId', 'visitStartTime', 'date',
       'totals', 'trafficSource', 'device', 'geoNetwork', 'customDimensions',
       'hits', 'fullVisitorId', 'userId', 'clientId', 'channelGrouping',
       'socialEngagementType'],
      dtype='object')


In [ ]:
df.to_csv("sample_ga_data500k_test.csv", index=False)

In [27]:
df_read_ga = pd.read_csv("sample_ga_data500K.csv")
df_read_ga.head()

,date,transactionRevenue,pageviews,visits,browser,country,source,hitNumber,fullVisitorId,visitId,visitNumber
0,20170216,33590000,8,1,Chrome,United States,(direct),1,3519741939483828661,1487262855,3
1,20170216,33590000,8,1,Chrome,United States,(direct),2,3519741939483828661,1487262855,3
2,20170216,33590000,8,1,Chrome,United States,(direct),3,3519741939483828661,1487262855,3
3,20170216,33590000,8,1,Chrome,United States,(direct),4,3519741939483828661,1487262855,3
4,20170216,33590000,8,1,Chrome,United States,(direct),5,3519741939483828661,1487262855,3


In [28]:
df_read_ga['date'] = pd.to_datetime(df_read_ga['date'], format="%Y%m%d") #'20170216'
df_ga_views  = df_read_ga.groupby(['date'])['pageviews'].sum().reset_index()

#Line plot of page views over time 

In [29]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df_ga_views['date'], y= df_ga_views['pageviews'], mode='lines', name = "Time Series"))
fig.update_layout(title = "Page Views over Time", xaxis_title='Date', yaxis_title="Page Views")
fig.show()

# Line plot of visits 

In [31]:
df_ga_visits = df_read_ga.groupby(['date'])['visits'].sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ga_visits['date'], y=df_ga_visits['visits'], mode='lines', name='Time Series'))
fig.update_layout(title="Visits over time", xaxis_title='Date', yaxis_title='visits')
fig.show()

# Histograms of Page Views & Visits

In [32]:
fig = px.histogram(df_ga_visits, x="visits")
fig.show()

In [33]:
fig = px.histogram(df_ga_views, x="pageviews")
fig.show()

# Pie chart of visits by country 

In [36]:
df_visits_country = df_read_ga.groupby(['country'])['visits'].sum().reset_index()
df_visits_country.loc[df_visits_country['visits'] <500 , 'country' ] = 'Other Countries'
fig = px.pie(df_visits_country, values='visits', names='country', title='Visits by Country')
fig.show()

#Training Cat Boost Model 

In [38]:
df_read_ga = pd.read_csv("sample_ga_data500K.csv")
df_read_ga.head()

,date,transactionRevenue,pageviews,visits,browser,country,source,hitNumber,fullVisitorId,visitId,visitNumber
0,20170216,33590000,8,1,Chrome,United States,(direct),1,3519741939483828661,1487262855,3
1,20170216,33590000,8,1,Chrome,United States,(direct),2,3519741939483828661,1487262855,3
2,20170216,33590000,8,1,Chrome,United States,(direct),3,3519741939483828661,1487262855,3
3,20170216,33590000,8,1,Chrome,United States,(direct),4,3519741939483828661,1487262855,3
4,20170216,33590000,8,1,Chrome,United States,(direct),5,3519741939483828661,1487262855,3


In [39]:
df_read_ga['date'] = pd.to_datetime(df_read_ga['date'], format='%Y%m%d')
df_read_ga['day'] = df_read_ga['date'].dt.day
df_read_ga['month'] = df_read_ga['date'].dt.month
df_read_ga['week'] = df_read_ga['date'].dt.week

<ipython-input-39-6ca487f4d07d>:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



In [40]:
df_read_ga.head()

,date,transactionRevenue,pageviews,visits,browser,country,source,hitNumber,fullVisitorId,visitId,visitNumber,day,month,week
0,2017-02-16,33590000,8,1,Chrome,United States,(direct),1,3519741939483828661,1487262855,3,16,2,7
1,2017-02-16,33590000,8,1,Chrome,United States,(direct),2,3519741939483828661,1487262855,3,16,2,7
2,2017-02-16,33590000,8,1,Chrome,United States,(direct),3,3519741939483828661,1487262855,3,16,2,7
3,2017-02-16,33590000,8,1,Chrome,United States,(direct),4,3519741939483828661,1487262855,3,16,2,7
4,2017-02-16,33590000,8,1,Chrome,United States,(direct),5,3519741939483828661,1487262855,3,16,2,7


In [42]:
df_train = df_read_ga[df_read_ga['month'] < 6 ].copy()
df_test = df_read_ga[df_read_ga['month'] > 6 ].copy()

X_train = df_train[['pageviews', 'browser', 'visits', 'week', 'month', 'day', 'fullVisitorId']]
X_test = df_test[['pageviews', 'browser', 'visits', 'week', 'month', 'day', 'fullVisitorId']]

y_train = df_train['transactionRevenue']
y_test = df_test['transactionRevenue']

cats = ['browser', 'fullVisitorId']

model = CatBoostRegressor(iterations=1000, cat_features=cats, random_state=42, loss_function='MAE')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)



0:	learn: 178917464.2738199	total: 140ms	remaining: 2m 19s
1:	learn: 177198951.2095488	total: 231ms	remaining: 1m 55s
2:	learn: 175619075.1616012	total: 303ms	remaining: 1m 40s
3:	learn: 174221790.5121061	total: 385ms	remaining: 1m 35s
4:	learn: 172873098.5718505	total: 460ms	remaining: 1m 31s
5:	learn: 171650251.6950943	total: 549ms	remaining: 1m 30s
6:	learn: 170355579.4171805	total: 620ms	remaining: 1m 27s
7:	learn: 169206058.5211292	total: 693ms	remaining: 1m 25s
8:	learn: 168158727.8065100	total: 772ms	remaining: 1m 24s
9:	learn: 167234746.0907140	total: 851ms	remaining: 1m 24s
10:	learn: 166325404.4768003	total: 936ms	remaining: 1m 24s
11:	learn: 165445793.2742789	total: 1.01s	remaining: 1m 23s
12:	learn: 164639136.7969284	total: 1.08s	remaining: 1m 21s
13:	learn: 163820863.3148900	total: 1.15s	remaining: 1m 20s
14:	learn: 163025079.4355865	total: 1.21s	remaining: 1m 19s
15:	learn: 162278661.8778685	total: 1.29s	remaining: 1m 19s
16:	learn: 161547203.1680915	total: 1.37s	remainin

In [44]:
from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error: ", mean_absolute_error(y_pred, y_test))

Mean Absolute Error:  126279759.99806525


#Plot Actual vs Predicted

In [52]:
import plotly.offline as pyo
df_read_ga = pd.read_csv("sample_ga_data500K.csv")
df_read_ga.head()
df_read_ga['date'] = pd.to_datetime(df_read_ga['date'], format='%Y%m%d')
df_read_ga['day'] = df_read_ga['date'].dt.day
df_read_ga['month'] = df_read_ga['date'].dt.month
df_read_ga['week'] = df_read_ga['date'].dt.week

df_train = df_read_ga[df_read_ga['month'] < 6 ].copy()
df_test = df_read_ga[df_read_ga['month'] > 6 ].copy()

df_test['predicted_revenue'] = y_pred
df_read_ga = df_read_ga.groupby(['date'])['transactionRevenue'].sum().reset_index()
df_test = df_test.groupby(['date'])['predicted_revenue'].sum().reset_index()

# Create a line trace for actual values
actual_trace = go.Scatter(
    x=list(df_read_ga['date']),
    y=df_read_ga['transactionRevenue'],
    name='Actual Values'
)

# Create a line trace for predicted values
predicted_trace = go.Scatter(
    x=list(df_test['date']),
    y=df_test['predicted_revenue'],
    name='Predicted Values'
)




# Create a layout for the chart
layout = go.Layout(
    title='Actual vs. Predicted Transaction Revenue',
    xaxis=dict(title='Time'),
    yaxis=dict(title='Value')
)

# Combine the traces and layout into a figure object
figure = go.Figure(data=[actual_trace, predicted_trace], layout=layout)

# Show the chart in a Jupyter notebook or in a separate browser window
pyo.iplot(figure)

<ipython-input-52-d94d665b7788>:7: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



#Plot feature importance 

In [53]:
feature_importances = model.get_feature_importance()

# Create a horizontal bar chart for feature importances
importance_trace = go.Bar(
    x=feature_importances,
    y=model.feature_names_,
    orientation='h',
    marker=dict(color='#4C78A8')
)

# Create a layout for the chart
layout = go.Layout(
    title='CatBoost Feature Importance',
    xaxis=dict(title='Importance'),
    yaxis=dict(title='Feature')
)

# Combine the trace and layout into a figure object
figure = go.Figure(data=[importance_trace], layout=layout)

# Show the chart in a Jupyter notebook or in a separate browser window
pyo.iplot(figure)

#Build Prophet model on COIVD Data 

In [55]:
query2 = """
SELECT * 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data` 
LIMIT 10 
"""
df = pandas_gbq.read_gbq(query2, project_id = project_id)
df.head()

Downloading: 100%|██████████|


,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,RO,2021-07-22,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
1,RO,2021-06-11,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
2,RO,2020-08-15,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
3,RO,2021-02-24,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
4,RO,2021-09-03,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)


In [56]:
print(list(df.columns))

['location_key', 'date', 'place_id', 'wikidata_id', 'datacommons_id', 'country_code', 'country_name', 'iso_3166_1_alpha_2', 'iso_3166_1_alpha_3', 'aggregation_level', 'new_confirmed', 'new_deceased', 'cumulative_confirmed', 'cumulative_deceased', 'cumulative_tested', 'new_persons_vaccinated', 'cumulative_persons_vaccinated', 'new_persons_fully_vaccinated', 'cumulative_persons_fully_vaccinated', 'new_vaccine_doses_administered', 'cumulative_vaccine_doses_administered', 'population', 'population_male', 'population_female', 'population_rural', 'population_urban', 'population_density', 'human_development_index', 'population_age_00_09', 'population_age_10_19', 'population_age_20_29', 'population_age_30_39', 'population_age_40_49', 'population_age_50_59', 'population_age_60_69', 'population_age_70_79', 'population_age_80_and_older', 'gdp_usd', 'gdp_per_capita_usd', 'openstreetmap_id', 'latitude', 'longitude', 'area_sq_km', 'smoking_prevalence', 'diabetes_prevalence', 'infant_mortality_rate',

In [57]:
query2 = """
SELECT 
  location_key,
  date,
  place_id,
  wikidata_id,
  datacommons_id,
  country_code,
  country_name,
  iso_3166_1_alpha_2,
  iso_3166_1_alpha_3,
  aggregation_level,
  new_confirmed,
  new_deceased,
  cumulative_confirmed,
  cumulative_deceased,
  cumulative_tested,
  new_persons_vaccinated,
  cumulative_persons_vaccinated,
  new_persons_fully_vaccinated,
  cumulative_persons_fully_vaccinated,
  new_vaccine_doses_administered,
  cumulative_vaccine_doses_administered,
FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE country_name IN ('United States of America', 'Italy', 'Spain', 'China', 'England') LIMIT 10
        """
df = pandas_gbq.read_gbq(query2, project_id = project_id)
df.head()

Downloading: 100%|██████████|


,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,new_deceased,cumulative_confirmed,cumulative_deceased,cumulative_tested,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered
0,US_AR,2021-03-06,ChIJYSc_dD-e0ocR0NLf_z5pBaQ,Q1612,geoId/05,US,United States of America,US,USA,1,...,14,324653,5297,2733062,24512,477127,7859,265226,39771,744139
1,US_AZ,2020-10-19,ChIJaxhMy-sIK4cRcc3Bf7EnOUI,Q816,geoId/04,US,United States of America,US,USA,1,...,3,231897,5830,2536524,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,US_AZ,2020-12-07,ChIJaxhMy-sIK4cRcc3Bf7EnOUI,Q816,geoId/04,US,United States of America,US,USA,1,...,0,365843,6950,4065296,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,US_CA,2021-06-27,ChIJPV4oX_65j4ARVW8IJ6IJUYs,Q99,geoId/06,US,United States of America,US,USA,1,...,31,3816045,63586,<NA>,66350,24083469,146676,19508119,208910,42312077
4,US_CA,2020-11-07,ChIJPV4oX_65j4ARVW8IJ6IJUYs,Q99,geoId/06,US,United States of America,US,USA,1,...,73,956957,17939,19565151,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [86]:
df_covid = pd.read_csv("sample_covid_data.csv")
df_covid.head()
df_covid_italy = df_covid[df_covid['country_name'] == "Italy"]

In [79]:
df_covid_model = df_covid[['cumulative_confirmed', 'date']]
df_covid_model = df_covid_model.groupby('date')['cumulative_confirmed'].sum().reset_index()

df_covid_model['ds'] = df_covid_model['date']
df_covid_model = df_covid_model.iloc[1:]
df_covid_model['ds'] = pd.to_datetime(df_covid_model['ds'])
df_covid_model['y'] = df_covid_model['cumulative_confirmed']
del df_covid_model['date']
del df_covid_model['cumulative_confirmed']
df_covid_model = df_covid_model[~df_covid_model['y'].isna()]
df_covid_model = df_covid_model[~df_covid_model['ds'].isna()]
df_covid_model = df_covid_model[df_covid_model['y'] > 0]

df_covid_model['y'] = np.log(df_covid_model['y'])
df_covid_model.reset_index(drop=True, inplace = True)



In [61]:
print(df_covid_model['ds'].max())
print(df_covid_model['ds'].min())

2022-09-15 00:00:00
2020-01-02 00:00:00


In [80]:
train = df_covid_model[df_covid_model['ds'] <= pd.to_datetime('2021-01-15')]

test = df_covid_model[(df_covid_model['ds'] > pd.to_datetime('2021-01-15'))&(df_covid_model['ds'] < pd.to_datetime('2021-01-31'))]

In [81]:
model = Prophet()
model.fit(train)
future = model.make_future_dataframe(periods=len(test))
forecast = model.predict(future)
forecast.head()

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpl_pi1d8h/9izs_51b.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpl_pi1d8h/5r1sa2e1.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=86431', 'data', 'file=/tmp/tmpl_pi1d8h/9izs_51b.json', 'init=/tmp/tmpl_pi1d8h/5r1sa2e1.json', 'output', 'file=/tmp/tmpl_pi1d8h/prophet_modelp2zrqh47/prophet_model-20230329183337.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:33:37 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:33:37 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-02,2.179881,0.593070,3.237413,2.179881,2.179881,-0.296938,-0.296938,-0.296938,-0.296938,-0.296938,-0.296938,0.0,0.0,0.0,1.882943
1,2020-01-05,2.526892,1.383463,3.897641,2.526892,2.526892,0.156123,0.156123,0.156123,0.156123,0.156123,0.156123,0.0,0.0,0.0,2.683015
2,2020-01-06,2.642562,1.445782,4.027182,2.642562,2.642562,0.105370,0.105370,0.105370,0.105370,0.105370,0.105370,0.0,0.0,0.0,2.747932
3,2020-01-07,2.758232,1.370833,3.872368,2.758232,2.758232,-0.132449,-0.132449,-0.132449,-0.132449,-0.132449,-0.132449,0.0,0.0,0.0,2.625783
4,2020-01-08,2.873903,1.573433,4.089072,2.873903,2.873903,-0.037370,-0.037370,-0.037370,-0.037370,-0.037370,-0.037370,0.0,0.0,0.0,2.836532


In [82]:
forecast['yhat'] = np.exp(forecast['yhat'])
train['y'] =  np.exp(train['y'])
test['y'] =  np.exp(test['y'])
forecast = forecast[(forecast['ds'] > pd.to_datetime('2021-01-15'))&(forecast['ds'] < pd.to_datetime('2021-01-31'))]

<ipython-input-82-9eeca0c19bec>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-82-9eeca0c19bec>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [83]:
fig = go.Figure()
# Add a line trace for the time series data

fig.add_trace(go.Scatter(x=train['ds'], y=train['y'], mode='lines', name='training'))
fig.add_trace(go.Scatter(x=test['ds'], y=test['y'], mode='lines', name='testing'))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='predictions'))

# Set the chart title and axis labels
fig.update_layout(title='Prophet: Actual Vs Predicted', xaxis_title='Date', yaxis_title='Covid Cummulatve Cases')

# Show the chart
fig.show()